In [7]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY_1010")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = "tech4impact_1010"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"


OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65072623-2467-476d-b1c2-b1289c29387d-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

Starts from now

In [ ]:
import os
import json
import getpass
import logging 
# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

    from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA



In [16]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [17]:
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # 총 초
    minutes = seconds // 60  # 분 계산
    remaining_seconds = seconds % 60  # 남은 초 계산
    return f"{minutes}분 {remaining_seconds}초"  # 출력 형식

# 파일 
with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']

# 발화를 Document 객체로 생성
documents = []
for utterance in utterances:
    content = utterance['msg']
    metadata = {
        'start_at': utterance['start_at'],  # 밀리초 단위
        'duration': utterance['duration'],   # 밀리초 단위
        'speaker': utterance['spk'],
        'speaker_type': utterance['spk_type']
    }
    documents.append(Document(page_content=content, metadata=metadata))
logging.info(f"총 {len(documents)}개의 문서 생성.")

# 시작 시간 기준으로 정렬
documents = sorted(documents, key=lambda doc: doc.metadata['start_at'])


2024-10-31 16:13:52,899 - INFO - 총 900개의 문서 생성.


In [ ]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"

# 언어 모델 초기화 # mini 버전 써보기
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)



# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=None)

# 리트리버 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


2024-10-31 16:13:59,063 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
import re
# 역할 지정해서 돌려보기
#role description
#확정은 결과물 가지고 펠로우님이랑 이야기
#chunk 늘려보기..
prompt = """
        당신은 전문적인 상담 세션을 분석하고 요약하는 AI입니다. 아래 상담 세션의 전체 스크립트를 주제별로 시간 순서에 따라 요약해 주세요. 다음 지침을 반드시 따르세요:
        
        1. 각 시간 구간은 **하나의 주요 질문** 또는 **하나의 주요 조언**으로 구성합니다. 하나의 구간에 여러 주제가 섞이지 않도록 주의해 주세요.
        2. 각 주제에 대해 시작 시간과 종료 시간을 명시합니다. 시작 시간은 해당 주제가 스크립트에서 처음 등장한 시점, 종료 시간은 마지막으로 등장한 시점을 기준으로 합니다.
        3. 각 요약은 다음 형식 중 **하나**로 작성합니다:
           - 질문인 경우: **Q.**으로 시작하여 질문 내용을 요약합니다.
           - 행동 변경이 필요한 경우: **[바꾸세요]**로 시작하여 변경이 필요한 행동을 요약합니다.
           - 권장 사항인 경우: **[이렇게 하세요]**로 시작하여 권장 사항을 요약합니다.
        
        **답변 예시:**
        **0분 42초 ~ 3분 15초**
        - Q. 오메가3 가 정말 유익한 거 맞나요?
        
        **5분 0초 ~ 7분 30초**
        - [바꾸세요] 당뇨 약은 중단하지 마세요.
        
        **10분 15초 ~ 12분 45초**
        - [이렇게 하세요] 약 복용을 지속하세요.
        
        **스크립트:**
        {text}
        
        **요약:**
"""

human_prompt = PromptTemplate(
    input_variables=[],
    template="요약:"
)


rag_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # 또는 "map_reduce" 등 필요에 따라 변경 가능
            retriever=retriever,
            return_source_documents=True
        )

# 전체 스크립트 요약 
response = rag_chain({"query": prompt.format(text=full_text)})
session_summary = response["result"]
source_documents = response["source_documents"]

print("=== Session Summary ===")
print(session_summary)
# print("\n=== Source Documents ===")
# for doc in source_documents:
#     print(f"Source: {doc.page_content}\n")


C:\Users\yooji\AppData\Local\Temp\ipykernel_21700\1209514253.py:46: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = rag_chain({"query": prompt.format(text=full_text)})
2024-10-31 16:16:02,776 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:16:21,494 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Session Summary ===
**0분 0초 ~ 1분 50초**
- Q. 당뇨약을 줄이는 게 현실적인가요?

**2분 5초 ~ 4분 7초**
- [이렇게 하세요] 당뇨 수치가 안정되면 스테로이드 사용을 줄이세요.

**5분 15초 ~ 6분 16초**
- [이렇게 하세요] 근력 운동을 통해 근육량을 늘리세요.

**7분 10초 ~ 8분 3초**
- Q. 비염에 아연과 프로폴리스가 효과가 있나요?

**10분 24초 ~ 11분 18초**
- [이렇게 하세요] 피로 회복을 위해 비타민 B군을 강화하세요.

**13분 20초 ~ 14분 5초**
- Q. 눈이 감기고 무거운 느낌이 드는데, 어떻게 해야 하나요?

**15분 8초 ~ 16분 26초**
- [이렇게 하세요] 오메가 3를 복용하여 눈의 뻑뻑함을 줄이세요.

**17분 8초 ~ 17분 28초**
- Q. 식물성 오메가 3와 동물성 오메가 3의 차이는 무엇인가요?

**18분 19초 ~ 19분 4초**
- [이렇게 하세요] 피로의 원인을 찾고, 항산화제를 복용하세요.

**20분 0초 ~ 20분 9초**
- [이렇게 하세요] 하체 운동을 통해 하지정맥류를 예방하세요.

**25분 28초 ~ 26분 25초**
- [이렇게 하세요] 30대 후반부터는 종합비타민과 오메가 3를 복용하세요.

**36분 22초 ~ 37분 0초**
- Q. 오메가 3가 건강에 유익한가요?

**40분 6초 ~ 41분 6초**
- [이렇게 하세요] 비타민 D 결핍 여부를 검사하고, 필요시 보충하세요.

**45분 8초 ~ 46분 8초**
- Q. 비타민 C와 D를 함께 복용해도 되나요?

**51분 8초 ~ 52분 3초**
- Q. 간염 보균자로서 약 복용 시 주의할 점은 무엇인가요?

**56분 5초 ~ 57분 0초**
- [이렇게 하세요] 약 복용 간격을 24시간으로 유지하세요.

**62분 22초 ~ 63분 15초**
- Q. 한약이 간에 영향을 주나요?

**66분 8초 ~ 67분 4초**
- Q. 고지혈증 약을 끊어도

In [28]:
# 출력 파싱, 주제와 시간 정보 추출
subtopics = re.findall(r'\*\*(\d+분 \d+초) ~ (\d+분 \d+초)\*\*\n- (.+?)(?=\n\*\*\d+분|\Z)', session_summary, re.DOTALL)

print(subtopics)


[('0분 0초', '1분 50초', 'Q. 당뇨약을 줄이는 게 현실적인가요?\n'), ('2분 5초', '4분 7초', '[이렇게 하세요] 당뇨 수치가 안정되면 스테로이드 사용을 줄이세요.\n'), ('5분 15초', '6분 16초', '[이렇게 하세요] 근력 운동을 통해 근육량을 늘리세요.\n'), ('7분 10초', '8분 3초', 'Q. 비염에 아연과 프로폴리스가 효과가 있나요?\n'), ('10분 24초', '11분 18초', '[이렇게 하세요] 피로 회복을 위해 비타민 B군을 강화하세요.\n'), ('13분 20초', '14분 5초', 'Q. 눈이 감기고 무거운 느낌이 드는데, 어떻게 해야 하나요?\n'), ('15분 8초', '16분 26초', '[이렇게 하세요] 오메가 3를 복용하여 눈의 뻑뻑함을 줄이세요.\n'), ('17분 8초', '17분 28초', 'Q. 식물성 오메가 3와 동물성 오메가 3의 차이는 무엇인가요?\n'), ('18분 19초', '19분 4초', '[이렇게 하세요] 피로의 원인을 찾고, 항산화제를 복용하세요.\n'), ('20분 0초', '20분 9초', '[이렇게 하세요] 하체 운동을 통해 하지정맥류를 예방하세요.\n'), ('25분 28초', '26분 25초', '[이렇게 하세요] 30대 후반부터는 종합비타민과 오메가 3를 복용하세요.\n'), ('36분 22초', '37분 0초', 'Q. 오메가 3가 건강에 유익한가요?\n'), ('40분 6초', '41분 6초', '[이렇게 하세요] 비타민 D 결핍 여부를 검사하고, 필요시 보충하세요.\n'), ('45분 8초', '46분 8초', 'Q. 비타민 C와 D를 함께 복용해도 되나요?\n'), ('51분 8초', '52분 3초', 'Q. 간염 보균자로서 약 복용 시 주의할 점은 무엇인가요?\n'), ('56분 5초', '57분 0초', '[이렇게 하세요] 약 복용 간격을 24시간으로 유지하세요.\n'), ('62분 22초', '63분 15초', 'Q. 한약이 간에 영향을 주

In [30]:
def time_str_to_ms(time_str):
    match = re.match(r'(\d+)분 (\d+)초', time_str)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return (minutes * 60 + seconds) * 1000
    else:
        logging.warning(f"시간 형식이 일치하지 않습니다: {time_str}")
        return 0
    
json_results = []
        
        # 주제별 루프 처리
for idx, (start_time_str, end_time_str, content) in enumerate(subtopics, 1):
    subtopic = content.strip()
    start_time_ms = time_str_to_ms(start_time_str)
    end_time_ms = time_str_to_ms(end_time_str)
    
    # 해당 시간 범위 내에서 retrieve
    retrieved_docs = retriever.get_relevant_documents(subtopic)
    
    relevant_docs = [
        doc for doc in retrieved_docs
        if start_time_ms <= doc.metadata['start_at'] <= end_time_ms
    ]
    
    # 관련 스크립트 생성
    related_scripts = [
        {
            "time": ms_to_minutes_seconds_str(doc.metadata['start_at']),
            "content": doc.page_content
        } for doc in relevant_docs
    ]
    
    # 주제별 데이터를 딕셔너리로 저장
    json_result = {
        "topic_id": idx,
        "start_time": start_time_str,
        "end_time": end_time_str,
        "content": subtopic,
        "related_scripts": related_scripts
    }
    json_results.append(json_result)
print(json_results)

2024-10-31 16:27:25,807 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:26,259 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:26,589 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:26,995 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:27,325 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:27,650 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:27,989 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:30,414 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:30,920 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-31 16:27:31,227 - INFO - HTTP

[{'topic_id': 1, 'start_time': '0분 0초', 'end_time': '1분 50초', 'content': 'Q. 당뇨약을 줄이는 게 현실적인가요?', 'related_scripts': [{'time': '1분 24초', 'content': '그래서 사실 당뇨약을 줄이는 게 제일 현실적인 것 같아요. 이게 뭔가? 그래서 그만큼 지금 이 약을.'}, {'time': '1분 24초', 'content': '그래서 사실 당뇨약을 줄이는 게 제일 현실적인 것 같아요. 이게 뭔가? 그래서 그만큼 지금 이 약을.'}]}, {'topic_id': 2, 'start_time': '2분 5초', 'end_time': '4분 7초', 'content': '[이렇게 하세요] 당뇨 수치가 안정되면 스테로이드 사용을 줄이세요.', 'related_scripts': [{'time': '3분 31초', 'content': '그래가지고 스테로이드를 쓰면은 좀 문제가 있다.'}, {'time': '3분 31초', 'content': '그래가지고 스테로이드를 쓰면은 좀 문제가 있다.'}, {'time': '2분 28초', 'content': '그 맨 처음에 이제 스트레이드 조사서 맞았는데 방뇨 수치가 되게 올라간다.'}, {'time': '2분 28초', 'content': '그 맨 처음에 이제 스트레이드 조사서 맞았는데 방뇨 수치가 되게 올라간다.'}]}, {'topic_id': 3, 'start_time': '5분 15초', 'end_time': '6분 16초', 'content': '[이렇게 하세요] 근력 운동을 통해 근육량을 늘리세요.', 'related_scripts': [{'time': '5분 15초', 'content': '그리고 당뇨 관리는 사실 장기적으로 봤을 때 근육량의 싸움입니다. 예, 그래서 꼭 근력 운동을 해주셔야 돼요.'}, {'time': '5분 15초', 'content': '그리고 당뇨 관리는 사실 장기적으로 봤을 때 근육량의 싸움입니다.

In [31]:
print(json_results[0])

{'topic_id': 1, 'start_time': '0분 0초', 'end_time': '1분 50초', 'content': 'Q. 당뇨약을 줄이는 게 현실적인가요?', 'related_scripts': [{'time': '1분 24초', 'content': '그래서 사실 당뇨약을 줄이는 게 제일 현실적인 것 같아요. 이게 뭔가? 그래서 그만큼 지금 이 약을.'}, {'time': '1분 24초', 'content': '그래서 사실 당뇨약을 줄이는 게 제일 현실적인 것 같아요. 이게 뭔가? 그래서 그만큼 지금 이 약을.'}]}


Next steps:

to create docker and upload to lambda,
docker buildx create --use
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
docker tag tenten_ai_topic:latest 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest
docker push 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest

In [ ]:
If you use docker buildx, make sure to reuse the builder instance rather than creating a new one every time:
bash
Copy code
# Create the builder only once
docker buildx create --use --name mybuilder
Then, you can use the existing builder for subsequent builds:
bash
Copy code
docker buildx use mybuilder
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
